In [7]:
import time
import json
import os

import torch
import numpy as np
import sympy as sp
from parsers import get_parser

import symbolicregression
from symbolicregression.envs import build_env
from symbolicregression.model import build_modules
from symbolicregression.trainer import Trainer
from symbolicregression.e2e_model import Transformer, pred_for_sample_no_refine, respond_to_batch , pred_for_sample, refine_for_sample, pred_for_sample_test, refine_for_sample_test 
from dyna_gym.agents.uct import UCT
from dyna_gym.agents.mcts import update_root, convert_to_json, print_tree
from rl_env import RLEnv
from default_pi import E2EHeuristic, NesymresHeuristic
from symbolicregression.metrics import compute_metrics


from nesymres.src.nesymres.architectures.model import Model
from nesymres.utils import load_metadata_hdf5
from nesymres.dclasses import FitParams, NNEquation, BFGSParams
from functools import partial
from sympy import lambdify
from reward import compute_reward_e2e, compute_reward_nesymres
import omegaconf

In [3]:
from argparse import Namespace
params = Namespace(backbone_model='e2e', seed=23, width=3, horizon=200, rollout=3, num_beams=1, train_value=False, no_seq_cache=True, no_prefix_cache=True, sample_only=False, ucb_constant=1.0, ucb_base=10.0, uct_alg='uct', ts_mode='best', alg='mcts', entropy_weighted_strategy='none', dump_path='', refinements_types='method=BFGS_batchsize=256_metric=/_mse', eval_dump_path=None, save_results=True, exp_name='debug', print_freq=100, save_periodic=25, exp_id='', fp16=False, amp=-1, rescale=True, embedder_type='LinearPoint', emb_emb_dim=64, enc_emb_dim=512, dec_emb_dim=512, n_emb_layers=1, n_enc_layers=2, n_dec_layers=16, n_enc_heads=16, n_dec_heads=16, emb_expansion_factor=1, n_enc_hidden_layers=1, n_dec_hidden_layers=1, norm_attention=False, dropout=0, attention_dropout=0, share_inout_emb=True, enc_positional_embeddings=None, dec_positional_embeddings='learnable', env_base_seed=0, test_env_seed=1, batch_size=1, batch_size_eval=64, optimizer='adam_inverse_sqrt,warmup_updates=10000', lr=1e-05, clip_grad_norm=0.5, n_steps_per_epoch=3000, max_epoch=100000, stopping_criterion='', accumulate_gradients=1, num_workers=10, train_noise_gamma=0.0, ablation_to_keep=None, max_input_points=200, n_trees_to_refine=10, export_data=False, reload_data='', reload_size=-1, batch_load=False, env_name='functions', queue_strategy=None, collate_queue_size=2000, use_sympy=False, simplify=False, use_abs=False, operators_to_downsample='div_0,arcsin_0,arccos_0,tan_0.2,arctan_0.2,sqrt_5,pow2_3,inv_3', operators_to_not_repeat='', max_unary_depth=6, required_operators='', extra_unary_operators='', extra_binary_operators='', extra_constants=None, min_input_dimension=1, max_input_dimension=10, min_output_dimension=1, max_output_dimension=1, enforce_dim=True, use_controller=True, float_precision=3, mantissa_len=1, max_exponent=100, max_exponent_prefactor=1, max_token_len=0, tokens_per_batch=10000, pad_to_max_dim=True, max_int=10, min_binary_ops_per_dim=0, max_binary_ops_per_dim=1, max_binary_ops_offset=4, min_unary_ops=0, max_unary_ops=4, min_op_prob=0.01, max_len=200, min_len_per_dim=5, max_centroids=10, prob_const=0.0, reduce_num_constants=True, use_skeleton=False, prob_rand=0.0, max_trials=1, n_prediction_points=200, tasks='functions', beam_eval=True, max_generated_output_len=200, beam_eval_train=0, beam_size=1, beam_type='sampling', beam_temperature=0.1, beam_length_penalty=1, lam=0.1, beam_early_stopping=True, beam_selection_metrics=1, max_number_bags=10, reload_model='', reload_checkpoint='', validation_metrics='r2_zero,r2,accuracy_l1_biggio,accuracy_l1_1e-3,accuracy_l1_1e-2,accuracy_l1_1e-1,_complexity', debug_train_statistics=False, eval_noise_gamma=0.0, eval_size=10000, eval_noise_type='additive', eval_noise=0, eval_only=False, eval_from_exp='', eval_data='', eval_verbose=0, eval_verbose_print=False, eval_input_length_modulo=-1, eval_on_pmlb=False, eval_mcts_on_pmlb=False, eval_in_domain=False, eval_mcts_in_domain=False, debug_slurm=False, debug=True, cpu=False, local_rank=-1, gpu_to_use='0', master_port=-1, windows=False, nvidia_apex=False, max_src_len=200, max_target_len=200, reward_type='nmse', reward_coef=1, vf_coef=0.0001, target_kl=1, entropy_coef=0.01, kl_regularizer=0.001, warmup_epoch=5, lr_patience=100, save_model=True, save_eval_dic='./eval_result', update_modules='all', actor_lr=1e-06, critic_lr=1e-05, kl_coef=0.01, rl_alg='ppo', run_id=1, pmlb_data_type='feynman', target_noise=0.0, prediction_sigmas='1,2,4,8,16')

In [6]:
import numpy as np
import torch
np.random.seed(params.seed)
torch.manual_seed(params.seed)
torch.cuda.manual_seed(params.seed)
params.debug = True
params.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
equation_env = build_env(params)
modules = build_modules(equation_env, params)
if not params.cpu:
    assert torch.cuda.is_available()
symbolicregression.utils.CUDA = not params.cpu
trainer = Trainer(modules, equation_env, params)

Checkpoint path does not exist, checkpoint.pth


In [11]:
model = Transformer(params = params, env=equation_env)
model.to(params.device) 

TypeError: Transformer.__init__() missing 1 required positional argument: 'samples'

In [9]:
from evaluate import evaluate_pmlb

# Basic usage
scores = evaluate_pmlb(
    trainer,          # Your model trainer instance
    params,          # Model parameters
    model,           # Your trained model
    target_noise=0.0,    # Amount of noise to add to target values
    random_state=29910,  # Random seed
    verbose=False,       # Whether to show detailed output
    save=True,          # Whether to save results
    save_suffix="./eval_result/my_evaluation.csv"  # Where to save results
)

NameError: name 'model' is not defined